
# Machine Learning Class 3: Neural Networks & Deep Learning

Welcome to the third and final class in our machine learning journey!
So far, we’ve learned how:

- Linear models uncover simple trends in data 📈
- Decision trees ask smart questions to classify and predict 🌳

Today, we turn to the most powerful tool in modern AI:

### ✨ Neural Networks — systems inspired by the brain that can learn almost anything.

### 🎯 What You'll Learn Today

1. **Neural Network Basics** — How artificial neurons learn from data
2. **Building Your First Network** — Step-by-step hands-on demo
3. **Going Deeper** — Why “deep” learning is so powerful
4. **Real Applications** — From image recognition to language translation
5. **Bonus** — Learn how to build the same network in Keras and PyTorch

### 🧠 **The Core Idea**

Neural networks are made of layers of tiny computing units — neurons — that:

- Take inputs (like pixel values or text features)
- Apply weights and nonlinear activations
- Pass results forward to make predictions

With enough neurons and layers, these networks can:

- Recognize complex images 🖼️
- Understand language and context 🌍
- Learn intricate patterns that no human could code by hand

### 🔗 Building on Previous Classes

| Class | Focus | Key Idea |
|-------|-------|----------|
| 1️⃣ Linear Models | Fit lines to patterns | Models assume a simple structure |
| 2️⃣ Trees | Ask yes/no questions | Let data dictate structure |
| 3️⃣ Neural Nets | Learn anything | Model learns structure from scratch |

Neural networks **don’t need hand-coded rules**. They build their own understanding by adjusting internal parameters based on data — making them the foundation of everything from ChatGPT to self-driving cars.

### 🌍 Why Neural Networks Matter

They power many of the tools you use daily:

- 📸 Image classifiers on your phone
- 🧠 Large Language Models like ChatGPT
- 🎧 Music & movie recommendations
- 🚗 Autonomous vehicles
- 🧬 Healthcare diagnostics

And they’re just getting started.

---

Let’s dive in and build one from scratch — then see how to train it using real-world libraries like TensorFlow/Keras and PyTorch.

Get ready to enter the world of deep learning! 🚀

In [ ]:
# Quick Setup - Import Our Neural Network Tools

import numpy as np
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
from sklearn.datasets import make_circles, load_digits
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow import keras
from tensorflow.keras import layers, Input
from ipywidgets import interact, FloatSlider, IntSlider

In [ ]:
# Set random seed for reproducible results
np.random.seed(42)


## Part 1: Neural Networks - Artificial Brains

### 🧠 **How Does Your Brain Work?**

Your brain has ~86 billion neurons that:
1. **Receive signals** from other neurons
2. **Process information** by combining signals
3. **Send output** to other neurons if activated
4. **Learn** by strengthening important connections

**Artificial neural networks** are a *simplified version* of this idea:

- They use numbers instead of electric signals.
- They "learn" by changing weights through data and feedback.

### 🔗 **From Biological to Artificial**

Let's break down what a single **artificial neuron** does:

| Component | Description |
|-----------|-------------|
| **Inputs** | Numbers representing features (like age, income, etc.) |
| **Weights** | How important each input is (learned during training) |
| **Activation** | Decides whether to "fire" based on weighted inputs |
| **Output** | A number passed to the next layer |

A **neural network** simply connects many of these neurons in layers:

- **Input layer** takes your data
- **Hidden layers** do the processing
- **Output layer** gives the final prediction

### 🎯 Why Networks Beat Single Neurons

- A single neuron can only draw a straight line — it’s like linear regression.
- A network with more neurons and layers can:
  - Bend, curve, and twist boundaries 🌀
  - Combine patterns from multiple inputs
  - Capture non-linear structures in data

Let’s build some intuition with a real example. 📊

### 🔍 Example: Can a Single Neuron Predict Who Will Buy?

We’ll use a tiny **customer dataset** with just two inputs: `age` and `income`.

Each customer either bought a product (1) or did not (0).

In [ ]:
customer_data = pd.DataFrame({
    'age': [25, 35, 45, 55, 30, 40, 50, 28, 38, 48],
    'income': [30, 50, 70, 90, 40, 60, 80, 35, 55, 75],
    'will_buy': [0, 0, 1, 1, 0, 1, 1, 0, 0, 1]
})
print(customer_data)

### 📈 Let's Visualize!

In [ ]:
fig = px.scatter(customer_data, x='age', y='income', color='will_buy',
                color_discrete_map={0: 'red', 1: 'green'},
                title="🛒 Customer Purchase Patterns",
                labels={'will_buy': 'Will Buy'})
fig.show()

### 🧪 Testing a Single Neuron

Let’s simulate how one artificial neuron would handle a single customer. Let's assume the custom is 40 years old with an income of $60,000.

In [ ]:
def simple_neuron(age, income, weight_age=0.1, weight_income=0.05, bias=-3):
    weighted_sum = weight_age * age + weight_income * income + bias

    # Let's use a sigmoid activation function
    activation = 1 / (1 + np.exp(-weighted_sum))
    prediction = 1 if activation > 0.5 else 0

    return prediction, activation

prediction, confidence = simple_neuron(age=40, income=60)
print(f"Prediction: {'Will Buy' if prediction == 1 else 'Will Not Buy'}, Confidence: {confidence:.2f}")

### 🎮 Interactive: Design Your Own Neuron

The following code defines a helper function to visualize how a single artificial neuron makes predictions.  
Don’t worry if it looks long — it's just setting up the plotting and interaction.

Now, use the sliders below to adjust the weights and bias and see how your neuron performs!  
Can you find a setting that gets most predictions right?

In [ ]:
@interact(
    weight_age=FloatSlider(value=0.1, min=-0.5, max=0.5, step=0.01, description='Weight (Age)'),
    weight_income=FloatSlider(value=0.05, min=-0.5, max=0.5, step=0.01, description='Weight (Income)'),
    bias=FloatSlider(value=-3.0, min=-10, max=10, step=0.1, description='Bias')
)
def design_neuron(weight_age, weight_income, bias):
    """Interactive tool to visualize a single neuron's decision boundary"""
    predictions = []
    confidences = []

    for _, row in customer_data.iterrows():
        pred, conf = simple_neuron(row['age'], row['income'], weight_age, weight_income, bias)
        predictions.append(pred)
        confidences.append(conf)

    accuracy = sum(p == t for p, t in zip(predictions, customer_data['will_buy'])) / len(predictions)

    # Set up plot
    fig = make_subplots(
        rows=1, cols=2,
        subplot_titles=("Neuron Predictions", "Decision Boundary"),
        specs=[[{"type": "scatter"}, {"type": "contour"}]]
    )

    # Left plot: correct (green) vs wrong (red) predictions
    colors = ['green' if p == t else 'red' for p, t in zip(predictions, customer_data['will_buy'])]

    fig.add_trace(
        go.Scatter(
            x=customer_data['age'],
            y=customer_data['income'],
            mode='markers',
            marker=dict(size=12, color=colors),
            name='Predictions'
        ),
        row=1, col=1
    )

    # Right plot: decision boundary via contour
    age_range = np.linspace(20, 60, 50)
    income_range = np.linspace(20, 100, 50)
    Age, Income = np.meshgrid(age_range, income_range)

    Z = np.zeros_like(Age)
    for i in range(Age.shape[0]):
        for j in range(Age.shape[1]):
            _, conf = simple_neuron(Age[i, j], Income[i, j], weight_age, weight_income, bias)
            Z[i, j] = conf

    fig.add_trace(
        go.Contour(
            x=age_range,
            y=income_range,
            z=Z,
            colorscale='RdYlGn',
            contours=dict(start=0, end=1, size=0.1),
            showscale=False,
            name='Decision Boundary'
        ),
        row=1, col=2
    )

    # Actual data points on decision plot
    fig.add_trace(
        go.Scatter(
            x=customer_data['age'],
            y=customer_data['income'],
            mode='markers',
            marker=dict(size=10, color=customer_data['will_buy'],
                        colorscale='RdYlGn', line=dict(width=2, color='black')),
            name='Actual Data'
        ),
        row=1, col=2
    )

    fig.update_layout(
        height=400,
        title_text=f"🧠 Your Neuron: {accuracy:.1%} Accuracy"
    )
    fig.update_xaxes(title_text="Age", row=1, col=1)
    fig.update_yaxes(title_text="Income ($k)", row=1, col=1)
    fig.update_xaxes(title_text="Age", row=1, col=2)
    fig.update_yaxes(title_text="Income ($k)", row=1, col=2)

    fig.show()

    # Console summary
    print(f"🧠 Your Neuron Performance:")
    print(f"   Accuracy: {accuracy:.1%}")
    print(f"   Weights: Age={weight_age:.2f}, Income={weight_income:.3f}")
    print(f"   Bias: {bias:.2f}")

    if accuracy >= 0.8:
        print("\n🎉 Excellent! Your neuron learned the pattern well!")
    elif accuracy >= 0.6:
        print("\n✅ Good! Try adjusting weights for better performance.")
    else:
        print("\n🤔 Keep experimenting with the weights and bias!")

### What This Does:

- It shows you how well your weights perform.
- The first plot shows which predictions were right (green) or wrong (red).
- The second plot shows your neuron’s decision boundary: where it predicts “yes” vs “no”.

### Interpreting the Output:

- A boundary that cuts through the data well means your neuron learned something useful!
- A boundary that misses many points means the neuron is too simple — or poorly tuned.
- Look at the accuracy printed — aim for at least 80%!

🧠 This helps you build intuition for:
- What weights and bias do
- How decision boundaries work
- Why real neural networks need more neurons and more layers

### 💡 Key Takeaways

- A single neuron is powerful but limited.
- It struggles with curved or complex boundaries.
- You just discovered why we need multi-layer neural networks — to handle complex patterns in data!

👉 Next, we’ll move from one neuron to an actual network, and see how it improves performance dramatically.

---

## Part 2: Building Your First Neural Network

### 🔄 From a Single Neuron to a Learning Machine

A single neuron can draw a line — but it can’t capture curves, corners, or complex decision boundaries.  
To move beyond that, we need **many neurons** working together in **layers**.

This is the core idea of a **neural network**.

### 🧱 What Happens Inside a Neural Network?

- The **input layer** passes features to the network
- **Hidden layers** process information step by step:
  - The first layer might detect simple signals
  - The next layer combines them into patterns
  - Later layers build more abstract understanding
- The **output layer** makes the final prediction

### 🌟 Why Is This Powerful?

- **Multiple neurons** allow flexible, curved decision boundaries  
- **Multiple layers** allow abstraction: from pixels → edges → shapes → faces  
- **Activation functions** give the network its non-linear magic

With the right architecture, a neural network can learn just about anything.

Let’s build one!


### 🌀 Step 1: Create a Challenging Dataset

We’ll now create a dataset that **cannot be solved with straight lines**.  
It has circular patterns — something only neural networks can untangle.

In [ ]:
# Create a more complex dataset that needs a neural network
X, y = make_circles(n_samples=300, noise=0.1, factor=0.3, random_state=42)

# Convert to DataFrame for easier handling
circle_data = pd.DataFrame(X, columns=['x', 'y'])
circle_data['class'] = y

print("🎯 Complex Pattern Dataset:")
print(f"   Data points: {len(circle_data)}")
print(f"   Classes: {len(np.unique(y))}")

### 🔍 Step 2: Visualize the Problem

Let’s plot the data to see why this is tricky.  
The two classes form **concentric circles** — a single line won’t separate them.

In [ ]:
# Visualize the complex pattern
fig = px.scatter(circle_data, x='x', y='y', color='class',
                color_discrete_map={0: 'red', 1: 'blue'},
                title="🎯 Complex Pattern: Circles within Circles")

fig.update_layout(height=400)
fig.show()

🧠 **Observation**:  
- ❌ **Linear models** will fail here  
- ❌ **Single neurons** won’t help  
- ✅ **Neural networks** are up for the challenge!


### 🛠️ Step 3: Prepare the Data

We’ll now split the data and scale it.  
**Why scale?** Neural networks are sensitive to the scale of input values.

In [ ]:
# Prepare data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Scale the data (important for neural networks)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

### 🤖 Step 4: Build and Train a Neural Network

Let’s create a real neural network with:
- **2 hidden layers**
- **10 neurons each**
- **ReLU activation**
- Up to **1000 training iterations**

In [ ]:
# Build a simple neural network with Keras
model = keras.Sequential([
    Input(shape=(2,)),
    layers.Dense(10, activation='relu'),
    layers.Dense(10, activation='relu'),
    layers.Dense(1, activation='sigmoid')
])

# Compile the model
model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy']
)

# Train the model and store training history
history = model.fit(X_train_scaled, y_train, epochs=100, verbose=0, validation_data=(X_test_scaled, y_test))

### 📈 Step 5: Evaluate Performance

Now let’s test the network’s accuracy and see how well it learned the pattern.

In [ ]:
# Evaluate on training and test sets
train_loss, train_accuracy = model.evaluate(X_train_scaled, y_train, verbose=0)
test_loss, test_accuracy = model.evaluate(X_test_scaled, y_test, verbose=0)

# Report results
print(f"\n🧠 Neural Network Results:")
print(f"   Training Accuracy: {train_accuracy:.1%}")
print(f"   Test Accuracy: {test_accuracy:.1%}")
print(f"   Network Architecture: Input → 10 → 10 → Output")


🎉 Great job! You've just trained a real neural network on a dataset that would stump simpler models.

### 📊 Step 7: Visualizing the Neural Network's Performance

Now that we've trained our neural network using TensorFlow/Keras, let’s see what it actually learned.

We'll look at:
- The decision boundary: where the model predicts one class vs another
- The loss curves: how the model improved during training

In [ ]:
# Create meshgrid for decision boundary
h = 0.02
x_min, x_max = X[:, 0].min() - 1, X[:, 0].max() + 1
y_min, y_max = X[:, 1].min() - 1, X[:, 1].max() + 1
xx, yy = np.meshgrid(np.arange(x_min, x_max, h),
                     np.arange(y_min, y_max, h))

# Predict over the grid
grid_points = np.c_[xx.ravel(), yy.ravel()]
grid_points_scaled = scaler.transform(grid_points)
Z = model.predict(grid_points_scaled, verbose=0)
Z = Z.reshape(xx.shape)

# Create subplots
fig = make_subplots(
    rows=1, cols=2,
    subplot_titles=('Neural Network Decision Boundary', 'Training Loss Curve'),
    specs=[[{"type": "contour"}, {"type": "xy"}]]
)

# Subplot 1: Decision boundary
fig.add_trace(
    go.Contour(
        x=np.arange(x_min, x_max, h),
        y=np.arange(y_min, y_max, h),
        z=Z,
        colorscale='RdYlBu',
        showscale=False,
        contours=dict(start=0, end=1, size=0.05),
        opacity=0.6,
    ),
    row=1, col=1
)

point_colors = ['red' if label == 0 else 'blue' for label in y]

fig.add_trace(
    go.Scatter(
        x=X[:, 0], y=X[:, 1],
        mode='markers',
        marker=dict(
            color=point_colors,
            line=dict(width=1, color='black'),
            size=8
        ),
        name='Data Points'
    ),
    row=1, col=1
)

# Subplot 2: Training and validation loss
fig.add_trace(
    go.Scatter(
        x=list(range(1, len(history.history['loss']) + 1)),
        y=history.history['loss'],
        mode='lines',
        name='Training Loss',
        line=dict(width=3, color='green')
    ),
    row=1, col=2
)

fig.add_trace(
    go.Scatter(
        x=list(range(1, len(history.history['val_loss']) + 1)),
        y=history.history['val_loss'],
        mode='lines',
        name='Validation Loss',
        line=dict(width=3, dash='dash', color='blue')
    ),
    row=1, col=2
)

# Layout
fig.update_layout(
    height=500,
    title_text=f"🧠 Neural Network: Decision Boundary and Loss Curve (Test Accuracy: {test_accuracy:.1%})",
    showlegend=True
)

fig.update_xaxes(title_text="X1", row=1, col=1)
fig.update_yaxes(title_text="X2", row=1, col=1)
fig.update_xaxes(title_text="Epoch", row=1, col=2)
fig.update_yaxes(title_text="Loss", row=1, col=2)

fig.show()


### 💬 What Can We Conclude?

- The **decision boundary** shows that the neural network learned to separate the circular classes quite well — something **linear models could never do**.
- The **loss curves** confirm that the model was able to reduce error during training and generalize well to unseen data (validation loss stays low).

Neural networks **automatically learn complex patterns** through multiple layers and nonlinear activations — that’s their power!

---

## 🎮 Explore Neural Networks Visually (Optional)

Want to **play with neurons, layers, and activations** without writing code?

👉 Head over to the [**TensorFlow Playground**](https://playground.tensorflow.org/)!

You can:
- Add or remove hidden layers
- Try different activation functions
- Tune learning rates and regularization
- Watch how the network adjusts to complex patterns like **concentric circles**

🔍 **Tip:** In the Playground, use the dataset with two circles (bottom-right option).  
It's the same one we just trained on!

This interactive tool is a perfect way to **see** what neural networks are learning — in real time.


---

## 🖼️ Deep Learning in Action: Recognizing Handwritten Digits

We’ve seen how neural networks can learn curved shapes — but can they recognize **images**?

In this example, we’ll use real image data:
- Each image is just **8×8 grayscale pixels**
- Each one shows a handwritten **digit (0–9)**
- Your goal: build a deep neural network to **recognize digits automatically**

This is a **mini-version of image recognition** and shows how deep learning powers tasks like OCR (optical character recognition).

In [ ]:
digits = load_digits()
X_digits, y_digits = digits.data, digits.target

print("🖼️ Handwritten Digit Recognition Dataset:")
print(f"   Images: {len(X_digits)}")
print(f"   Pixels per image: {X_digits.shape[1]} (8x8 grid)")
print(f"   Classes: {len(set(y_digits))} digits (0–9)")

### 👁️‍🗨️ Let’s Look at the Data

Each digit is represented by a flat list of 64 numbers (8×8 grayscale pixels). Let’s visualize some of them.


In [ ]:
# Show some example digits
fig = make_subplots(
    rows=2, cols=5,
    subplot_titles=[f'Digit {i}' for i in range(10)]
)

for i in range(10):
    idx = (y_digits == i).nonzero()[0][0]
    image = X_digits[idx].reshape(8, 8)

    row = 1 if i < 5 else 2
    col = (i % 5) + 1

    fig.add_trace(
        go.Heatmap(z=image, colorscale='gray', showscale=False),
        row=row, col=col
    )

fig.update_layout(height=400, title_text="🖼️ Sample Handwritten Digits (8x8 pixels)")
fig.show()


### 🧠 Step 1: Prepare the Data

We scale the pixel values (just like with the circles) and split the data into training and test sets.


In [ ]:
# Split and scale the data
X_train_digits, X_test_digits, y_train_digits, y_test_digits = train_test_split(
    X_digits, y_digits, test_size=0.3, random_state=42
)

scaler_digits = StandardScaler()
X_train_digits_scaled = scaler_digits.fit_transform(X_train_digits)
X_test_digits_scaled = scaler_digits.transform(X_test_digits)

### 🤖 Step 2: Build a Deep Neural Network

We now build a **deeper network** that can:
- Learn from all 64 input pixels
- Use **ReLU activations**
- Have **3 hidden layers** with decreasing size (100 → 50 → 25)
- Predict the correct digit (0–9) using a **softmax output layer**

In [ ]:
model_digits = keras.Sequential([
    layers.Input(shape=(64,)),  # 8x8 images flattened to 64 pixels
    layers.Dense(100, activation='relu'),
    layers.Dense(50, activation='relu'),
    layers.Dense(25, activation='relu'),
    layers.Dense(10, activation='softmax')  # 10 output classes
])

model_digits.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

history_digits = model_digits.fit(
    X_train_digits_scaled, y_train_digits,
    epochs=30, validation_data=(X_test_digits_scaled, y_test_digits),
    verbose=0
)

### 📈 Step 3: Check Accuracy

Let’s see how well the model performs on both training and test sets.


In [ ]:
train_loss, train_acc = model_digits.evaluate(X_train_digits_scaled, y_train_digits, verbose=0)
test_loss, test_acc = model_digits.evaluate(X_test_digits_scaled, y_test_digits, verbose=0)

print(f"🎯 Deep Learning Results:")
print(f"   Training Accuracy: {train_acc:.1%}")
print(f"   Test Accuracy: {test_acc:.1%}")

### 🔍 Step 4: Try the Network on New Digits

Let’s ask the model to classify some unseen digits — and show how confident it is.

In [ ]:
# Pick 5 random digits from the test set
np.random.seed(42)
sample_indices = np.random.choice(len(X_test_digits), size=5, replace=False)

for idx in sample_indices:
    true_label = y_test_digits[idx]
    image = X_test_digits[idx].reshape(1, -1)
    prediction = model_digits.predict(image, verbose=0)
    predicted_label = np.argmax(prediction)
    confidence = np.max(prediction)

    print(f"🔢 True: {true_label} | Predicted: {predicted_label} | Confidence: {confidence:.1%}")

### ✅ Summary: What You Learned

- Neural networks can go **beyond synthetic patterns** and handle real-world images.
- With enough depth and training, they can classify digits with **over 95% accuracy**!
- This is a simplified version of what powers real tools like OCR, CAPTCHA solvers, and postal scanners.

Next steps:
- Learn how **convolutional layers** can improve image recognition
- Explore more complex datasets like **MNIST** or **CIFAR-10**


## 🎮 Try It Yourself: Digit Classifier in Action

Use the slider below to explore how the neural network performs on different handwritten digits.

Watch:
- What it gets **right** ✅
- What it **misclassifies** ❌
- How **confident** it is in its answers 🔍

In [ ]:
@interact(
    digit_index=IntSlider(
        value=0,
        min=0,
        max=len(X_test_digits) - 1,
        step=1,
        description='Digit #:',
        continuous_update=False
    )
)
def test_digit_classifier_keras(digit_index):
    """Interactive digit classification tool using Keras model"""

    # Safeguard against out-of-bounds index
    if digit_index >= len(X_test_digits):
        digit_index = len(X_test_digits) - 1

    test_image = X_test_digits[digit_index]
    true_label = y_test_digits[digit_index]

    # Predict with Keras model
    input_scaled = X_test_digits_scaled[digit_index].reshape(1, -1)
    probabilities = model_digits.predict(input_scaled, verbose=0)[0]
    prediction = np.argmax(probabilities)

    # Create side-by-side plots
    fig = make_subplots(
        rows=1, cols=2,
        subplot_titles=('🖼️ Test Image', '📊 Network Prediction'),
        specs=[[{"type": "heatmap"}, {"type": "bar"}]]
    )

    # Left: Show the digit image
    fig.add_trace(
        go.Heatmap(z=test_image.reshape(8, 8), colorscale='gray', showscale=False),
        row=1, col=1
    )

    # Right: Bar chart of predicted probabilities
    digits = list(range(10))
    bar_colors = ['green' if i == prediction else 'lightblue' for i in digits]

    fig.add_trace(
        go.Bar(x=digits, y=probabilities, marker_color=bar_colors),
        row=1, col=2
    )

    # Add vertical line for the true label
    fig.add_vline(x=true_label, line_dash="dash", line_color="red",
                  annotation_text=f"True: {true_label}", row=1, col=2)

    # Prepare summary
    result = "✅ CORRECT" if prediction == true_label else "❌ WRONG"
    confidence = probabilities[prediction]

    fig.update_layout(
        height=400,
        title_text=f"🧠 Prediction: {prediction} (True: {true_label}) — {result} ({confidence:.1%} confidence)"
    )

    fig.update_xaxes(title_text="Pixel Position", row=1, col=1)
    fig.update_xaxes(title_text="Digit", row=1, col=2, tickmode='linear', dtick=1)
    fig.update_yaxes(title_text="Pixel Position", row=1, col=1)
    fig.update_yaxes(title_text="Probability", row=1, col=2)

    fig.show()

    print(f"🎯 Classification Summary:")
    print(f"   True Digit: {true_label}")
    print(f"   Predicted : {prediction}")
    print(f"   Confidence: {confidence:.1%}")
    print(f"   Result    : {result}")

    top_3 = np.argsort(probabilities)[-3:][::-1]
    print(f"\n🏆 Top 3 Predictions:")
    for rank, idx in enumerate(top_3, start=1):
        print(f"   {rank}. Digit {idx}: {probabilities[idx]:.1%}")

## 🧠 Bonus: Neural Networks in PyTorch

Keras (TensorFlow) is great for fast prototyping — but PyTorch offers flexibility and is widely used in research.

Here’s how you can build and train the same neural network using **PyTorch**.

We'll:
- Define a custom neural network class
- Use binary cross-entropy loss and the Adam optimizer
- Train the model and evaluate accuracy

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.metrics import accuracy_score

# Convert data to PyTorch tensors
X_train_tensor = torch.tensor(X_train_scaled, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train.reshape(-1, 1), dtype=torch.float32)
X_test_tensor = torch.tensor(X_test_scaled, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test.reshape(-1, 1), dtype=torch.float32)

# Define a simple feedforward neural network
class SimpleNN(nn.Module):
    def __init__(self):
        super(SimpleNN, self).__init__()
        self.net = nn.Sequential(
            nn.Linear(2, 10),
            nn.ReLU(),
            nn.Linear(10, 10),
            nn.ReLU(),
            nn.Linear(10, 1),
            nn.Sigmoid()
        )

    def forward(self, x):
        return self.net(x)

# Instantiate the model, define loss and optimizer
model = SimpleNN()
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

# Training loop
epochs = 100
for epoch in range(epochs):
    model.train()
    optimizer.zero_grad()
    outputs = model(X_train_tensor)
    loss = criterion(outputs, y_train_tensor)
    loss.backward()
    optimizer.step()

# Evaluate on test data
model.eval()
with torch.no_grad():
    test_preds = model(X_test_tensor).numpy()
    test_preds_class = (test_preds > 0.5).astype(int)

accuracy = accuracy_score(y_test, test_preds_class)

print(f"✅ PyTorch Test Accuracy: {accuracy:.1%}")

### 🔄 What’s Different?

- You define your own `nn.Module` class to specify the architecture
- Training uses a manual loop (gives you more control)
- PyTorch works well with **autograd** and **GPU acceleration**

Both Keras and PyTorch are excellent — the choice often depends on the task and your preference.

🧪 Try recreating this network for other datasets!

## 🎓 Congratulations – You've Completed the Machine Learning Series!

Over the past three sessions, you've built a strong foundation in machine learning – from simple models to deep neural networks powering modern AI.

---

### 🧭 Your Learning Journey

**🔹 Class 1: Linear Models**
- Discovered how machines learn from data with straight-line models  
- Understood gradient descent as a tool for optimization  
- Built the mathematical intuition behind model training  

**🔹 Class 2: Decision Trees & Ensembles**  
- Learned how models split data into interpretable rules  
- Saw how combining many trees creates powerful predictors (Random Forests)  
- Gained tools that balance accuracy and explainability  

**🔹 Class 3: Neural Networks & Deep Learning**  
- Explored how layered “neurons” can learn complex, non-linear patterns  
- Trained deep models to recognize patterns in visuals and data  
- Saw how AI powers modern tools like voice assistants, image recognition, and language translation  

---

### 💡 Big Takeaways

- 🧠 **No single model fits all problems** – know your tools  
- 🔍 **Complex patterns require complex models**, but they also need more data and care  
- ⚖️ **Simplicity vs. performance is always a trade-off**  
- 🔄 **Interpreting and validating results is as important as building the model**

---

### 🌐 Where Machine Learning Shows Up Around You

- 🎬 **Netflix recommendations** – collaborative filtering + neural nets  
- 📸 **Face recognition** – deep convolutional networks  
- ✉️ **Spam detection** – decision trees on text features  
- 🚗 **Self-driving perception** – computer vision + reinforcement learning  
- 🌍 **Real-time translation** – sequence models and transformers  

---

### 🚀 What's Next?

**🔧 Practice & Projects**
- Apply what you’ve learned to datasets from your field  
- Build your first real-world ML mini-project  
- Use platforms like Kaggle or Hugging Face to explore datasets and models  

**📚 Keep Learning**
- Dive into:
  - Computer Vision
  - Natural Language Processing
  - Reinforcement Learning
  - Generative AI  
- Try TensorFlow, PyTorch, Scikit-Learn in more depth

💼 **Start using machine learning in your field!**

---

### 🛠️ Final Wisdom

You don’t need a PhD to start using machine learning effectively.

What matters most:
- 🔍 **Knowing which model fits the problem**
- 🧪 **Testing, validating, and improving your model**
- 🧠 **Thinking critically about data and bias**
- 📢 **Explaining your results to others**

---

### 🌟 You’re Ready

This is just the beginning — ML is a journey of curiosity, experimentation, and creativity.

Go explore, build, ask questions, and make something amazing.  
We’re excited to see what you’ll do. 🚀🤖
